In [1]:
import numpy as np
from datasets import load_dataset

from transformers import (
    AutoTokenizer, 
    DataCollatorForTokenClassification,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    pipeline
)

import evaluate

c:\Users\rebal\ner-tuning\ner\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Download Dataset

In [2]:
wnut = load_dataset("wnut_17", trust_remote_code=True)

# Create NER Labels

In [3]:
label_list = wnut["train"].features[f"ner_tags"].feature.names
label_list

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

# Tokenize Data

In [4]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

example = wnut["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

In [5]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Create Evaluation Metrics

In [6]:
seqeval = evaluate.load("seqeval")
labels = [label_list[i] for i in example[f"ner_tags"]]

In [7]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [8]:
id2label = {
    0: "O",
    1: "B-corporation",
    2: "I-corporation",
    3: "B-creative-work",
    4: "I-creative-work",
    5: "B-group",
    6: "I-group",
    7: "B-location",
    8: "I-location",
    9: "B-person",
    10: "I-person",
    11: "B-product",
    12: "I-product",
}
label2id = {
    "O": 0,
    "B-corporation": 1,
    "I-corporation": 2,
    "B-creative-work": 3,
    "I-creative-work": 4,
    "B-group": 5,
    "I-group": 6,
    "B-location": 7,
    "I-location": 8,
    "B-person": 9,
    "I-person": 10,
    "B-product": 11,
    "I-product": 12,
}

# Train Model

In [9]:
model = AutoModelForTokenClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=13, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
training_args = TrainingArguments(
    output_dir="model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["validation"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.242733,0.690590,0.517943,0.591934,0.954606
2,No log,0.253904,0.699029,0.516746,0.594223,0.954797
3,0.052100,0.259952,0.683307,0.523923,0.593094,0.954924
4,0.052100,0.265385,0.692542,0.544258,0.609511,0.956132
5,0.022500,0.271296,0.691950,0.534689,0.603239,0.955687


TrainOutput(global_step=1065, training_loss=0.0361436664778302, metrics={'train_runtime': 82.4698, 'train_samples_per_second': 205.772, 'train_steps_per_second': 12.914, 'total_flos': 229914027537180.0, 'train_loss': 0.0361436664778302, 'epoch': 5.0})

In [19]:
trainer.save_model()

In [20]:
ner_pipeline = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

Device set to use cuda:0


In [21]:
text = "John lives in New York"
outputs = ner_pipeline(text)
for entity in outputs:
    print(f"{entity['word']} -> {entity['entity_group']} ({entity['score']:.2f})")

john -> person (0.97)
new york -> location (0.75)


In [23]:
raw_preds, labels, _ = trainer.predict(tokenized_wnut['test'])

In [44]:
predictions = np.argmax(raw_preds, axis=2)

true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

# Print some predictions
for i in range(5):
    print(i+1)
    toks = wnut['test'][i]['tokens']
    preds = list(zip(true_labels[i], true_predictions[i]))
    # print(preds)

    for a, (i, j) in enumerate(preds):
        if i != 'O':
            print(toks[a], '->', i)
    print()

1
Sonmarg -> B-location

2
Waltengoo -> B-location
Nar -> I-location

3

4

5
Avalanche -> B-group
Rescue -> I-group
Teams -> I-group
ART -> B-group

